In [1]:
import src.TX as TX
from src import MACGenerator, UDP_TX, SlidingBook
import numpy as np
import cv2
import time
import socket
import struct

def ProMAC_X(M, x):
    # Initialize an NxN matrix with zeros
    matrix = np.zeros((M, M), dtype=int)
    
    # Populate the matrix
    for i in range(M):
        # Determine the start position for the ones in the current row
        start_pos = i % M
        for j in range(x):
            col_pos = (start_pos + j) 
            if col_pos >= M:
                continue
            matrix[i, col_pos] = 1
    
    return matrix

#### parameters that needs to be exhanged between the sender and the receiver #####
                # t1  t2  t3  t4  t5  t6  t7  t8  t9
X = np.array([[ 1,  1,  0,  0,  0,  0,  0,  0,  0], # m1
              [ 1,  1,  0,  0,  0,  0,  0,  0,  0], # m2
              [ 0,  1,  1,  0,  0,  0,  0,  0,  0], # m3
              [ 0,  0,  1,  0,  0,  0,  0,  0,  0], # m4
              [ 0,  0,  1,  1,  0,  0,  0,  0,  0], # m5
              [ 0,  0,  0,  1,  0,  0,  0,  0,  0], # m6
              [ 0,  0,  0,  1,  1,  0,  0,  0,  0], # m7
              [ 0,  0,  0,  0,  1,  0,  0,  0,  0], # m8
              [ 0,  0,  0,  0,  1,  1,  0,  0,  0], # m9
              [ 0,  0,  0,  0,  0,  1,  0,  0,  0], # m10
              [ 0,  0,  0,  0,  0,  1,  1,  0,  0], # m11
              [ 0,  0,  0,  0,  0,  0,  1,  0,  0], # m12
              [ 0,  0,  0,  0,  0,  0,  1,  1,  0], # m13
              [ 0,  0,  0,  0,  0,  0,  0,  1,  0], # m14
              [ 0,  0,  0,  0,  0,  0,  0,  1,  1], # m15
              [ 0,  0,  0,  0,  0,  0,  0,  0,  1], # m16
              [ 0,  0,  0,  0,  0,  0,  0,  0,  1], # m17
              [ 0,  0,  0,  0,  0,  0,  0,  0,  1]]) # m18
 
                # t1  t2  t3  t4  t5  t6  t7  t8  t9
Y = np.array(  [[ 0,  0,  0,  0,  0,  0,  1,  0,  0], # m1
                [ 0,  0,  0,  0,  0,  0,  0,  1,  0], # m2
                [ 1,  0,  0,  0,  0,  0,  0,  0,  0], # m3
                [ 0,  0,  0,  0,  0,  0,  0,  0,  0], # m4
                [ 0,  0,  0,  0,  0,  0,  0,  0,  0], # m5
                [ 0,  1,  0,  0,  0,  0,  0,  0,  0], # m6
                [ 0,  0,  0,  0,  0,  0,  0,  0,  0], # m7
                [ 0,  0,  0,  0,  0,  0,  0,  0,  0], # m8
                [ 0,  0,  1,  0,  0,  0,  0,  0,  0], # m9
                [ 0,  0,  0,  0,  0,  0,  0,  0,  0], # m10
                [ 0,  0,  0,  0,  0,  0,  0,  0,  0], # m11
                [ 0,  0,  0,  1,  0,  0,  0,  0,  0], # m12
                [ 0,  0,  0,  0,  0,  0,  0,  0,  0], # m13
                [ 0,  0,  0,  0,  0,  0,  0,  0,  0], # m14
                [ 0,  0,  0,  0,  1,  0,  0,  0,  0], # m15
                [ 0,  0,  0,  0,  0,  0,  0,  0,  0], # m16
                [ 0,  0,  0,  0,  0,  1,  0,  0,  0], # m17
                [ 0,  0,  0,  0,  0,  0,  0,  0,  1]]) # m18
KEY = b"key"
DIGESTMOD = 'sha384' 
IP = "0.0.0.0"
PORT = 23422
Payload_Size_Byte = 64
quality = 70
attack_probability = 0.05

X = np.eye(18)
X = ProMAC_X(5, 2)
Y = np.eye(5)

buffer = SlidingBook(num_pages=100, page_size=X.shape[0])
page_processor = MACGenerator(X = X, Y = Y, secret_key=KEY, digestmod=DIGESTMOD)
udp_tx = UDP_TX(IP= IP, PORT= PORT, buffer=buffer, page_processor= page_processor, Payload_Size_Byte=Payload_Size_Byte)

with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.connect((IP, PORT))
    s.send(b"start")
    time.sleep(.1)
    data = time.time()
    data = struct.pack('d', data)
    s.send(data)



# with open("send.txt", "rb") as f:
#     text = f.read()
#     print(len(text))
# 1/0

cap = cv2.VideoCapture(0)  # Capture video from the default camera
# Define the compression parameters for Progressive JPEG
compression_params = [cv2.IMWRITE_JPEG_PROGRESSIVE, 1, cv2.IMWRITE_JPEG_QUALITY, quality]
## counting the number of frames per second
start_time = time.time()
frame_counter = 0
average_tx_size = 0
while True:
    ret, frame = cap.read()
    # Encode the image to JPEG format in memory
    success, encoded_frame = cv2.imencode('.jpg', frame, compression_params)
    data = encoded_frame.tobytes()
    average_tx_size+=len(data)//1000
    buffer = SlidingBook(num_pages=40, page_size=X.shape[0])
    page_processor = MACGenerator(X = X, Y = Y, secret_key=KEY, digestmod=DIGESTMOD)
    udp_tx = UDP_TX(IP= IP, PORT= PORT, buffer=buffer, page_processor= page_processor)
    pages = udp_tx.segment_to_pages(data)
    udp_tx.transmit(pages, attack_probability)
    cv2.imshow('Webcam', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):  # Press 'q' to quit
        break
    time.sleep(.01)
    frame_counter += 1
    if time.time() - start_time >= 1:
        print("frame per second: ", frame_counter, "fps ,data rate: ", average_tx_size, "KB/s")
        frame_counter = 0
        start_time = time.time()
        average_tx_size = 0
cap.release()



2024-09-03 18:02:17.376 Python[19245:345411] WARNING: Secure coding is automatically enabled for restorable state! However, not on all supported macOS versions of this application. Opt-in to secure coding explicitly by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState:.


frame per second:  8 fps ,data rate:  1009 KB/s
frame per second:  14 fps ,data rate:  1720 KB/s
frame per second:  16 fps ,data rate:  1932 KB/s
frame per second:  16 fps ,data rate:  1871 KB/s
frame per second:  15 fps ,data rate:  1815 KB/s
frame per second:  16 fps ,data rate:  1980 KB/s
frame per second:  15 fps ,data rate:  1765 KB/s
frame per second:  15 fps ,data rate:  1786 KB/s
frame per second:  15 fps ,data rate:  1778 KB/s
frame per second:  15 fps ,data rate:  1780 KB/s
frame per second:  16 fps ,data rate:  1920 KB/s


ConnectionRefusedError: [Errno 61] Connection refused

In [ ]:
import time

time.time()